#Building an image retrieval system with deep features


#Fire up GraphLab Create

In [16]:
import graphlab

#Load the CIFAR-10 dataset

We will use a popular benchmark dataset in computer vision called CIFAR-10.  

(We've reduced the data to just 4 categories = {'cat','bird','automobile','dog'}.)

This dataset is already split into a training set and test set. In this simple retrieval example, there is no notion of "testing", so we will only use the training data.

In [17]:
image_train = graphlab.SFrame('image_train_data/')

#Computing deep features for our images

The two lines below allow us to compute deep features.  This computation takes a little while, so we have already computed them and saved the results as a column in the data you loaded. 

(Note that if you would like to compute such deep features and have a GPU on your machine, you should use the GPU enabled GraphLab Create, which will be significantly faster for this task.)

In [18]:
#deep_learning_model = graphlab.load_model('http://s3.amazonaws.com/GraphLab-Datasets/deeplearning/imagenet_model_iter45')
#image_train['deep_features'] = deep_learning_model.extract_features(image_train)

In [19]:
image_train.head()

Columns:
	id	int
	image	Image
	label	str
	deep_features	array
	image_array	array

Rows: 10

Data:
+-----+----------------------+------------+-------------------------------+
|  id |        image         |   label    |         deep_features         |
+-----+----------------------+------------+-------------------------------+
|  24 | Height: 32 Width: 32 |    bird    | [0.242871761322, 1.0954537... |
|  33 | Height: 32 Width: 32 |    cat     | [0.525087952614, 0.0, 0.0,... |
|  36 | Height: 32 Width: 32 |    cat     | [0.566015958786, 0.0, 0.0,... |
|  70 | Height: 32 Width: 32 |    dog     | [1.12979578972, 0.0, 0.0, ... |
|  90 | Height: 32 Width: 32 |    bird    | [1.71786928177, 0.0, 0.0, ... |
|  97 | Height: 32 Width: 32 | automobile | [1.57818555832, 0.0, 0.0, ... |
| 107 | Height: 32 Width: 32 |    dog     | [0.0, 0.0, 0.220677852631,... |
| 121 | Height: 32 Width: 32 |    bird    | [0.0, 0.23753464222, 0.0, ... |
| 136 | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 0.0, 0.0, ... |
| 138 | Height: 32 Width: 32 |    bird    | [0.658935725689, 0.0, 0.0,... |
+-----+----------------------+------------+-------------------------------+
+-------------------------------+
|          image_array          |
+-------------------------------+
| [73.0, 77.0, 58.0, 71.0, 6... |
| [7.0, 5.0, 8.0, 7.0, 5.0, ... |
| [169.0, 122.0, 65.0, 131.0... |
| [154.0, 179.0, 152.0, 159.... |
| [216.0, 195.0, 180.0, 201.... |
| [33.0, 44.0, 27.0, 29.0, 4... |
| [97.0, 51.0, 31.0, 104.0, ... |
| [93.0, 96.0, 88.0, 102.0, ... |
| [35.0, 59.0, 53.0, 36.0, 5... |
| [205.0, 193.0, 195.0, 200.... |
+-------------------------------+
[10 rows x 5 columns]

#Train a nearest-neighbors model for retrieving images using deep features

We will now build a simple image retrieval system that finds the nearest neighbors for any image.

In [20]:
knn_model = graphlab.nearest_neighbors.create(image_train,features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.


#Use image retrieval model with deep features to find similar images

Let's find similar images to this cat picture.

In [21]:
graphlab.canvas.set_target('ipynb')
cat = image_train[18:19]
cat['image'].show()

In [22]:
knn_model.query(cat)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 11.9ms       |
PROGRESS: | Done         |         | 100         | 111.944ms    |
PROGRESS: +--------------+---------+-------------+--------------+


Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |       384       |      0.0      |  1   |
|      0      |       6910      | 36.9403137951 |  2   |
|      0      |      39777      | 38.4634888975 |  3   |
|      0      |      36870      | 39.7559623119 |  4   |
|      0      |      41734      | 39.7866014148 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]

We are going to create a simple function to view the nearest neighbors to save typing:

In [23]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

In [24]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 7.304ms      |
PROGRESS: | Done         |         | 100         | 111.346ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [25]:
cat_neighbors['image'].show()

Very cool results showing similar cats.

##Finding similar images to a car

In [26]:
car = image_train[8:9]
car['image'].show()

In [27]:
get_images_from_ids(knn_model.query(car))['image'].show()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 10.518ms     |
PROGRESS: | Done         |         | 100         | 109.414ms    |
PROGRESS: +--------------+---------+-------------+--------------+


#Just for fun, let's create a lambda to find and show nearest neighbor images

In [28]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_train[i:i+1]))['image'].show()

In [29]:
show_neighbors(8)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 10.387ms     |
PROGRESS: | Done         |         | 100         | 99.627ms     |
PROGRESS: +--------------+---------+-------------+--------------+


In [30]:
show_neighbors(26)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 7.081ms      |
PROGRESS: | Done         |         | 100         | 107.913ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [31]:
show_neighbors_test = lambda i: get_images_from_ids(knn_model.query(image_test[i:i+1]))['image'].show()

In [145]:
show_test  = lambda i: image_test[i:i+1]['image'].show()
show_image  = lambda i: image_train[i:i+1]['image'].show()

In [33]:
image_test = graphlab.SFrame('image_test_data/')

In [34]:
show_neighbors_test(1)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 7.223ms      |
PROGRESS: | Done         |         | 100         | 94.501ms     |
PROGRESS: +--------------+---------+-------------+--------------+


In [57]:
image = 10
show_test(image)
show_neighbors_test(image)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 10.337ms     |
PROGRESS: | Done         |         | 100         | 109.504ms    |
PROGRESS: +--------------+---------+-------------+--------------+


In [59]:
image_train['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


In [66]:
image_train_cat = image_train.filter_by("cat", 'label')
image_train_automobile  = image_train.filter_by("automobile", 'label')
image_train_dog  = image_train.filter_by("dog", 'label')
image_train_bird  = image_train.filter_by("bird", 'label')

In [67]:
cat_model = graphlab.nearest_neighbors.create(image_train_cat,features=['deep_features'],
                                             label='id')
dog_model = graphlab.nearest_neighbors.create(image_train_dog,features=['deep_features'],
                                             label='id')
bird_model = graphlab.nearest_neighbors.create(image_train_bird,features=['deep_features'],
                                             label='id')
automobile_model = graphlab.nearest_neighbors.create(image_train_automobile,features=['deep_features'],
                                             label='id')

PROGRESS: Starting brute force nearest neighbors model training.
PROGRESS: Starting brute force nearest neighbors model training.
PROGRESS: Starting brute force nearest neighbors model training.
PROGRESS: Starting brute force nearest neighbors model training.


In [69]:
cat = image_test[0:1]
cat["image"].show()

In [148]:
cat_model.query(cat)


PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 10.167ms     |
PROGRESS: | Done         |         | 100         | 38.7ms       |
PROGRESS: +--------------+---------+-------------+--------------+


Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |      16289      |  34.623719208 |  1   |
|      0      |      45646      | 36.0068799284 |  2   |
|      0      |      32139      | 36.5200813436 |  3   |
|      0      |      25713      | 36.7548502521 |  4   |
|      0      |       331       | 36.8731228168 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]

In [150]:
image_train.filter_by(16289 ,'id')['image'].show()

In [73]:
knn_model.query(cat)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.0498753   | 8.117ms      |
PROGRESS: | Done         |         | 100         | 92.998ms     |
PROGRESS: +--------------+---------+-------------+--------------+


Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |      16289      |  34.623719208 |  1   |
|      0      |      45646      | 36.0068799284 |  2   |
|      0      |      32139      | 36.5200813436 |  3   |
|      0      |      25713      | 36.7548502521 |  4   |
|      0      |       331       | 36.8731228168 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]

In [74]:
dog_model.query(cat)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 8.663ms      |
PROGRESS: | Done         |         | 100         | 38.057ms     |
PROGRESS: +--------------+---------+-------------+--------------+


Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |      16976      | 37.4642628784 |  1   |
|      0      |      13387      | 37.5666832169 |  2   |
|      0      |      35867      | 37.6047267079 |  3   |
|      0      |      44603      | 37.7065585153 |  4   |
|      0      |       6094      | 38.5113254907 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]

In [151]:
image_train.filter_by(16976 ,'id')['image'].show()

In [76]:
result = cat_model.query(cat)
result['distance'].mean()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 9.999ms      |
PROGRESS: | Done         |         | 100         | 40.364ms     |
PROGRESS: +--------------+---------+-------------+--------------+


36.15573070978294

In [77]:
result = dog_model.query(cat)
result['distance'].mean()

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.196464    | 9.241ms      |
PROGRESS: | Done         |         | 100         | 36.914ms     |
PROGRESS: +--------------+---------+-------------+--------------+


37.77071136184157

In [78]:
image_test_cat = image_test.filter_by("cat", 'label')
image_test_automobile  = image_test.filter_by("automobile", 'label')
image_test_dog  = image_test.filter_by("dog", 'label')
image_test_bird  = image_test.filter_by("bird", 'label')

In [86]:
dog_cat_neighbors = cat_model.query(image_test_dog, k=1)
dog_automobile_neighbors = automobile_model.query(image_test_dog, k=1)
dog_bird_neighbors = bird_model.query(image_test_dog, k=1)
dog_dog_neighbors = dog_model.query(image_test_dog, k=1)


PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 8
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 64000   | 12.5737     | 207.931ms    |
PROGRESS: | Done         | 509000  | 100         | 256.727ms    |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: Starting blockwise querying.
PROGRESS: max rows per data block: 7668
PROGRESS: number of reference data blocks: 8
PROGRESS: number of query data blocks: 1
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 1000         | 63000   | 12.3772     | 241.247ms    |
PROGRESS: 

In [87]:
dog_distances  = graphlab.SFrame({'dog-automobile': dog_automobile_neighbors['distance'],
                                  'dog-bird': dog_bird_neighbors['distance'],
                                  'dog-cat': dog_cat_neighbors['distance'],
                                  'dog-dog': dog_dog_neighbors['distance']
                                 })

In [88]:
dog_distances.head()


Columns:
	dog-automobile	float
	dog-bird	float
	dog-cat	float
	dog-dog	float

Rows: 10

Data:
+----------------+---------------+---------------+---------------+
| dog-automobile |    dog-bird   |    dog-cat    |    dog-dog    |
+----------------+---------------+---------------+---------------+
| 41.9579761457  | 41.7538647304 | 36.4196077068 | 33.4773590373 |
| 46.0021331807  | 41.3382958925 | 38.8353268874 | 32.8458495684 |
| 42.9462290692  | 38.6157590853 | 36.9763410854 | 35.0397073189 |
| 41.6866060048  | 37.0892269954 | 34.5750072914 | 33.9010327697 |
| 39.2269664935  |  38.272288694 |  34.778824791 | 37.4849250909 |
| 40.5845117698  | 39.1462089236 | 35.1171578292 |  34.945165344 |
| 45.1067352961  |  40.523040106 | 40.6095830913 | 39.0957278345 |
| 41.3221140974  | 38.1947918393 | 39.9036867306 | 37.7696131032 |
| 41.8244654995  | 40.1567131661 | 38.0674700168 | 35.1089144603 |
| 45.4976929401  | 45.5597962603 | 42.7258732951 | 43.2422832585 |
+----------------+---------------+---------------+---------------+
[10 rows x 4 columns]

In [140]:
def is_dog_correct(row):
    return ((row['dog-dog'] < row['dog-cat']) & (row['dog-dog'] < row['dog-bird']) & (row['dog-dog'] < row['dog-automobile']) )

In [142]:
is_dog_correct(dog_distances[4:5])

dtype: int
Rows: 1
[0]

In [144]:
dog_distances.apply(is_dog_correct).sum()

678